In [1]:
import re
import os
import time
import joblib 

import numpy as np
import pandas as pd
import scipy.sparse as sp

from datetime import datetime

dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

Revised on: 2021-01-20


In [2]:
import json
import urlextract
from nltk.stem import WordNetLemmatizer

def load_data(data):
    raw_path = os.path.join("data","1_raw")
    filename = ''.join([data, ".csv"])
    out_dfm = pd.read_csv(os.path.join(raw_path, filename))
    out_arr = np.array(out_dfm.iloc[:,0].ravel())
    return out_arr

X_train = load_data("X_train")
y_train = load_data("y_train")

y = y_train.copy()

# transform y_array into int type
y[y=='ham'] = 0
y[y=='spam'] = 1
y = y.astype('int')

# load contractions map for custom cleanup
with open("contractions_map.json") as f:
    contractions_map = json.load(f)

In [3]:
import custom.clean_preprocess as cp
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

pipe = Pipeline([('counter', cp.DocumentToNgramCounterTransformer(n_grams=3)),
                 ('bot', cp.WordCounterToVectorTransformer(vocabulary_size=2000)),
                 ('tfidf', TfidfTransformer(sublinear_tf=True))])

X_tfidf = pipe.fit_transform(X_train)

In [4]:
from scipy.sparse.linalg import svds
from sklearn.utils.extmath import svd_flip
from sklearn.preprocessing import MaxAbsScaler

def perform_SVD(X, n_components=300): 
    
    X_array = X.asfptype()
    U, Sigma, VT = svds(X_array.T, # term-document matrix
                        k=n_components)
    # reverse outputs
    Sigma = Sigma[::-1]
    U, VT = svd_flip(U[:, ::-1], VT[::-1])
    
    # return V 
    V = VT.T
    scaler = MaxAbsScaler()
    V_scaled = scaler.fit_transform(V)
    return V_scaled # scaled for Logistic Regression

X_tfidf_svd = perform_SVD(X_tfidf, n_components=800)

In [5]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

X_tfidf_svd_allcos = cosine_similarity(X_tfidf_svd)

train_df = pd.DataFrame({'sms':X_train, 'target':y_train})

# get spam indexes
spam_ix = train_df.loc[train_df['target']=='spam'].index

# calculate average spam similarity on SVD
mean_spam_sims = []

for ix in range(X_tfidf_svd_allcos.shape[0]):
    mean_spam_sims.append(np.mean(X_tfidf_svd_allcos[ix, spam_ix]))

X_tfidf_svd_spamcos = sp.hstack((csr_matrix(mean_spam_sims).T, X_tfidf_svd)) 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_tfidf_svd_spamcos,
                                                  y, 
                                                  stratify=y,
                                                  random_state=42)

In [8]:
# instantiate estimators
log_clf = LogisticRegression(
    solver="liblinear"
    , random_state=42
)

rnd_clf = RandomForestClassifier(
    n_jobs=-1
    , random_state=42
    , max_depth=8
    , max_features=150
    , min_samples_split=3
    , n_estimators=100
)

svm_clf = SVC(
    random_state=42
)

voting_clf = VotingClassifier(
    estimators=[('log', log_clf), ('rnd', rnd_clf), ('svm', svm_clf)],
    voting='hard'
)

In [12]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__, __name__, 'acc', accuracy_score(y_val, y_pred))
    print(clf.__class__, __name__, 'tpr', recall_score(y_val, y_pred, pos_label=1)) 
    print(clf.__class__, __name__, 'tnr', recall_score(y_val, y_pred, pos_label=0)) 

<class 'sklearn.linear_model._logistic.LogisticRegression'> __main__ acc 0.9723076923076923
<class 'sklearn.linear_model._logistic.LogisticRegression'> __main__ tpr 0.8062015503875969
<class 'sklearn.linear_model._logistic.LogisticRegression'> __main__ tnr 0.9976359338061466
<class 'sklearn.ensemble._forest.RandomForestClassifier'> __main__ acc 0.9887179487179487
<class 'sklearn.ensemble._forest.RandomForestClassifier'> __main__ tpr 0.937984496124031
<class 'sklearn.ensemble._forest.RandomForestClassifier'> __main__ tnr 0.9964539007092199
<class 'sklearn.svm._classes.SVC'> __main__ acc 0.958974358974359
<class 'sklearn.svm._classes.SVC'> __main__ tpr 0.689922480620155
<class 'sklearn.svm._classes.SVC'> __main__ tnr 1.0
<class 'sklearn.ensemble._voting.VotingClassifier'> __main__ acc 0.9743589743589743
<class 'sklearn.ensemble._voting.VotingClassifier'> __main__ tpr 0.8062015503875969
<class 'sklearn.ensemble._voting.VotingClassifier'> __main__ tnr 1.0


Even after scaling to help out the logistic classifier, this ensemble still does more poorly than the highly optimized random forest model, which already does more poorly than if we hadnt scaled the SVD.

In [13]:
for clf in (log_clf, rnd_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__, __name__, 'acc', accuracy_score(y_val, y_pred))
    print(clf.__class__, __name__, 'tpr', recall_score(y_val, y_pred, pos_label=1)) 
    print(clf.__class__, __name__, 'tnr', recall_score(y_val, y_pred, pos_label=0)) 

<class 'sklearn.linear_model._logistic.LogisticRegression'> __main__ acc 0.9723076923076923
<class 'sklearn.linear_model._logistic.LogisticRegression'> __main__ tpr 0.8062015503875969
<class 'sklearn.linear_model._logistic.LogisticRegression'> __main__ tnr 0.9976359338061466
<class 'sklearn.ensemble._forest.RandomForestClassifier'> __main__ acc 0.9887179487179487
<class 'sklearn.ensemble._forest.RandomForestClassifier'> __main__ tpr 0.937984496124031
<class 'sklearn.ensemble._forest.RandomForestClassifier'> __main__ tnr 0.9964539007092199
<class 'sklearn.ensemble._voting.VotingClassifier'> __main__ acc 0.9743589743589743
<class 'sklearn.ensemble._voting.VotingClassifier'> __main__ tpr 0.8062015503875969
<class 'sklearn.ensemble._voting.VotingClassifier'> __main__ tnr 1.0


__Morale__: the *wisdom of the crowds* only works when everyone in the crowd is more or less clueless and makes mistakes. When we have an "expert" in the crowd, we should probably follow that expert, and so an ensemble will not necessarily perform better. Even after removing the highly "nonoptimized" SVD classifier, the voting classifier still doesn't recognize the superiority of the expert.